## Kombinasi 5 :
- Delete duplicate
- Impute missing value using iterative imputer
- Outlier handling with log tranformation
- Encoding
- Standard Scaler
- Feature selection with Decision Tree

In [34]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [35]:
df = pd.read_csv("../UFC_train.csv")

df.head()

,R_fighter,B_fighter,Referee,date,location,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,Winner
0,Joe Riggs,Joe Doerksen,Steve Mazzagatti,2004-08-21,"Las Vegas, Nevada, USA",False,Middleweight,NaN,NaN,NaN,...,0,0,0,Southpaw,182.88,177.80,185.0,26.0,21.0,Red
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",False,Lightweight,1.156250,0.000000,0.394141,...,0,1,0,Orthodox,180.34,187.96,170.0,27.0,30.0,Blue
2,Dan Stittgen,Stephen Thompson,Josh Rosenthal,2012-02-04,"Las Vegas, Nevada, USA",False,Welterweight,NaN,NaN,NaN,...,0,0,0,Orthodox,185.42,NaN,170.0,28.0,31.0,Blue
3,Josh Koscheck,Johny Hendricks,Kevin Mulhall,2012-05-05,"East Rutherford, New Jersey, USA",False,Welterweight,0.695312,0.000000,0.783359,...,6,3,0,Orthodox,177.80,185.42,170.0,28.0,34.0,Blue
4,John Dodson,Manvel Gamburyan,James Warring,2016-04-16,"Tampa, Florida, USA",False,Bantamweight,0.500000,0.266602,0.381462,...,3,0,1,Orthodox,160.02,167.64,135.0,34.0,31.0,Red


In [36]:
# Delete duplicate
print(f"Shape before dropping duplicate : {df.shape}")
df.drop_duplicates(inplace=True)
print(f"Shape after dropping duplicate : {df.shape}")

Shape before dropping duplicate : (5410, 144)
Shape after dropping duplicate : (5410, 144)


In [37]:
def check_null(df):
    col_na = df.isnull().sum().sort_values(ascending=True)
    percent = col_na / len(df)
    missing_data = pd.concat([col_na, percent], axis=1, keys=['Total', 'Percent'])

    if (missing_data[missing_data['Total'] > 0].shape[0] == 0):
        print("Tidak ditemukan missing value pada dataset")
    else:
        print(missing_data[missing_data['Total'] > 0])

In [38]:
check_null(df)

                         Total   Percent
R_Weight_lbs                 2  0.000370
R_Height_cms                 4  0.000739
B_Weight_lbs                 8  0.001479
B_Height_cms                10  0.001848
R_Stance                    27  0.004991
...                        ...       ...
B_avg_GROUND_landed       1293  0.239002
B_avg_GROUND_att          1293  0.239002
B_avg_opp_CLINCH_landed   1293  0.239002
B_avg_TD_att              1293  0.239002
B_avg_HEAD_landed         1293  0.239002

[109 rows x 2 columns]


In [39]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

In [40]:
# Impute missing value using iterative imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.tree import ExtraTreeRegressor

imputer = IterativeImputer(random_state=42, max_iter=15, initial_strategy='median', estimator=ExtraTreeRegressor())
df_imputed_number = imputer.fit_transform(df_number)
df_imputed_number = pd.DataFrame(df_imputed_number, columns=df_number.columns)

/Users/azka/opt/anaconda3/lib/python3.9/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [41]:
# Drop R_fighter, B_fighter, Referee, location, date
df_object = df_object.drop(['R_fighter', 'B_fighter', 'Referee', 'location', 'date'], axis=1)

In [42]:
# Combine df_object and df_imputed_number
df = pd.concat([df_object, df_imputed_number], axis=1)
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Middleweight,Orthodox,Southpaw,Red,0.000000,1.000000,0.440000,0.620000,0.750000,0.165000,...,0.0,0.0,0.0,0.0,0.0,182.88,177.80,185.0,26.0,21.0
1,Lightweight,Orthodox,Orthodox,Blue,1.156250,0.000000,0.394141,0.352422,0.239219,0.011484,...,0.0,4.0,0.0,1.0,0.0,180.34,187.96,170.0,27.0,30.0
2,Welterweight,Orthodox,Orthodox,Blue,0.000000,0.000000,0.490000,0.622500,0.250000,0.754375,...,0.0,0.0,0.0,0.0,0.0,185.42,187.96,170.0,28.0,31.0
3,Welterweight,Southpaw,Orthodox,Blue,0.695312,0.000000,0.783359,0.185547,0.088281,0.104375,...,1.0,5.0,6.0,3.0,0.0,177.80,185.42,170.0,28.0,34.0
4,Bantamweight,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,0.0,2.0,3.0,0.0,1.0,160.02,167.64,135.0,34.0,31.0


In [43]:
check_null(df)

          Total   Percent
R_Stance     27  0.004991
B_Stance     64  0.011830


In [44]:
# Impute missing R_stance and B_stance with mode
df['R_Stance'].fillna(df['R_Stance'].mode()[0], inplace=True)
df['B_Stance'].fillna(df['B_Stance'].mode()[0], inplace=True)

In [45]:
check_null(df)

Tidak ditemukan missing value pada dataset


In [46]:
def check_outlier(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)

    # Menghitung RUB dan RLB.
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR

    # Menampilkan banyaknya outlier pada atribut.
    outliers = (df < lower_limit) | (df > upper_limit)
    print ("Outlier pada tiap atribut:")
    print(outliers.sum())

    return outliers

In [47]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

In [48]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

Outlier pada tiap atribut:
B_avg_KD                 560
B_avg_opp_KD             546
B_avg_SIG_STR_pct        142
B_avg_opp_SIG_STR_pct    120
B_avg_TD_pct               0
                        ... 
R_Height_cms              19
R_Reach_cms               81
R_Weight_lbs             301
B_age                     31
R_age                     74
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 10.35120147874307%
B_avg_opp_KD = 10.092421441774492%
B_avg_SIG_STR_pct = 2.6247689463955637%
B_avg_opp_SIG_STR_pct = 2.2181146025878005%
B_avg_TD_pct = 0.0%
B_avg_opp_TD_pct = 0.0%
B_avg_SUB_ATT = 7.486136783733826%
B_avg_opp_SUB_ATT = 7.208872458410351%
B_avg_REV = 11.756007393715343%
B_avg_opp_REV = 18.280961182994453%
B_avg_SIG_STR_att = 3.234750462107209%
B_avg_SIG_STR_landed = 2.865064695009242%
B_avg_opp_SIG_STR_att = 3.2717190388170057%
B_avg_opp_SIG_STR_landed = 2.975970425138632%
B_avg_TOTAL_STR_att = 1.977818853974122%
B_avg_TOTAL_STR_landed = 2.0147874306839

In [49]:
# Add a constant value to make all values positive
df_number_positive = df_number + abs(df_number.min()) + 1

df_number_log = np.log(df_number_positive)


In [50]:
df_number_log.head()

,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,B_avg_opp_REV,...,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,0.000000,0.693147,0.364643,0.482426,0.559616,0.152721,0.000000,0.000000,0.594707,0.559616,...,0.000000,0.000000,0.000000,0.000000,0.000000,5.817944,5.802722,5.707110,3.806662,3.713572
1,0.768371,0.000000,0.332278,0.301897,0.214481,0.011419,0.145182,0.124703,0.000000,0.000000,...,0.000000,1.609438,0.000000,0.693147,0.000000,5.810362,5.832938,5.655992,3.828641,3.912023
2,0.000000,0.000000,0.398776,0.483968,0.223144,0.562113,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,5.825469,5.832938,5.655992,3.850148,3.931826
3,0.527867,0.000000,0.578499,0.170204,0.084600,0.099280,0.089612,0.060625,0.000000,0.000000,...,0.693147,1.791759,1.945910,1.386294,0.000000,5.802722,5.825469,5.655992,3.850148,3.988984
4,0.405465,0.236337,0.323143,0.376076,0.357405,0.384970,0.866230,0.000244,0.117783,0.405465,...,0.000000,1.098612,1.386294,0.000000,0.693147,5.747544,5.771566,5.525453,3.970292,3.931826


In [51]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

Outlier pada tiap atribut:
B_avg_KD                 560
B_avg_opp_KD             546
B_avg_SIG_STR_pct        142
B_avg_opp_SIG_STR_pct    120
B_avg_TD_pct               0
                        ... 
R_Height_cms              19
R_Reach_cms               81
R_Weight_lbs             301
B_age                     31
R_age                     74
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 10.35120147874307%
B_avg_opp_KD = 10.092421441774492%
B_avg_SIG_STR_pct = 2.6247689463955637%
B_avg_opp_SIG_STR_pct = 2.2181146025878005%
B_avg_TD_pct = 0.0%
B_avg_opp_TD_pct = 0.0%
B_avg_SUB_ATT = 7.486136783733826%
B_avg_opp_SUB_ATT = 7.208872458410351%
B_avg_REV = 11.756007393715343%
B_avg_opp_REV = 18.280961182994453%
B_avg_SIG_STR_att = 3.234750462107209%
B_avg_SIG_STR_landed = 2.865064695009242%
B_avg_opp_SIG_STR_att = 3.2717190388170057%
B_avg_opp_SIG_STR_landed = 2.975970425138632%
B_avg_TOTAL_STR_att = 1.977818853974122%
B_avg_TOTAL_STR_landed = 2.0147874306839

In [52]:
df['gender'] = df['weight_class'].apply(lambda x: 'women' if 'women' in x.lower() else 'male')

In [53]:
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
0,Middleweight,Orthodox,Southpaw,Red,0.000000,1.000000,0.440000,0.620000,0.750000,0.165000,...,0.0,0.0,0.0,0.0,182.88,177.80,185.0,26.0,21.0,male
1,Lightweight,Orthodox,Orthodox,Blue,1.156250,0.000000,0.394141,0.352422,0.239219,0.011484,...,4.0,0.0,1.0,0.0,180.34,187.96,170.0,27.0,30.0,male
2,Welterweight,Orthodox,Orthodox,Blue,0.000000,0.000000,0.490000,0.622500,0.250000,0.754375,...,0.0,0.0,0.0,0.0,185.42,187.96,170.0,28.0,31.0,male
3,Welterweight,Southpaw,Orthodox,Blue,0.695312,0.000000,0.783359,0.185547,0.088281,0.104375,...,5.0,6.0,3.0,0.0,177.80,185.42,170.0,28.0,34.0,male
4,Bantamweight,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,2.0,3.0,0.0,1.0,160.02,167.64,135.0,34.0,31.0,male


In [54]:
df['weight_class'] = df['weight_class'].str.replace('Women', '')

In [55]:
# Get all weight_class values
weight_class = df['weight_class'].unique()
weight_class

array(['Middleweight', 'Lightweight', 'Welterweight', 'Bantamweight',
       'Flyweight', 'LightHeavyweight', 'Strawweight', 'Featherweight',
       'OpenWeight', 'Heavyweight', 'CatchWeight'], dtype=object)

In [56]:
# Create a dictionary to map weight_class values to numbers
weight_class_dict = {
    'CatchWeight' : 0,
    'Strawweight' : 1,
    'Flyweight' : 2,
    'Bantamweight' : 3,
    'Featherweight' : 4,
    'Lightweight' : 5,
    'Welterweight' : 6,
    'Middleweight' : 7,
    'LightHeavyweight' : 8,
    'Heavyweight' : 9,
    'OpenWeight' : 10,
}

gender_dict = {
    'male' : 1,
    'women' : 0
}

In [57]:
# Map each weight_class value to the correct number
df['weight_class'] = df['weight_class'].map(weight_class_dict)

In [58]:
# Map each gender value to the correct number
df['gender'] = df['gender'].map(gender_dict)

In [59]:
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
0,7,Orthodox,Southpaw,Red,0.000000,1.000000,0.440000,0.620000,0.750000,0.165000,...,0.0,0.0,0.0,0.0,182.88,177.80,185.0,26.0,21.0,1
1,5,Orthodox,Orthodox,Blue,1.156250,0.000000,0.394141,0.352422,0.239219,0.011484,...,4.0,0.0,1.0,0.0,180.34,187.96,170.0,27.0,30.0,1
2,6,Orthodox,Orthodox,Blue,0.000000,0.000000,0.490000,0.622500,0.250000,0.754375,...,0.0,0.0,0.0,0.0,185.42,187.96,170.0,28.0,31.0,1
3,6,Southpaw,Orthodox,Blue,0.695312,0.000000,0.783359,0.185547,0.088281,0.104375,...,5.0,6.0,3.0,0.0,177.80,185.42,170.0,28.0,34.0,1
4,3,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,2.0,3.0,0.0,1.0,160.02,167.64,135.0,34.0,31.0,1


In [60]:
# Get the remaining object columns
df_object = df.select_dtypes(include='object')

# Perform one-hot encoding
df_encoded = pd.get_dummies(df_object)

# Combine the encoded columns with the original dataframe
df = pd.concat([df, df_encoded], axis=1)

# Drop the original object columns
df.drop(df_object.columns, axis=1, inplace=True)

df.head()


,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,...,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Sideways,R_Stance_Southpaw,R_Stance_Switch,Winner_Blue,Winner_Draw,Winner_Red
0,7,0.000000,1.000000,0.440000,0.620000,0.750000,0.165000,0.00000,0.000000,0.8125,...,0,0,0,0,0,1,0,0,0,1
1,5,1.156250,0.000000,0.394141,0.352422,0.239219,0.011484,0.15625,0.132812,0.0000,...,0,0,0,1,0,0,0,1,0,0
2,6,0.000000,0.000000,0.490000,0.622500,0.250000,0.754375,0.00000,0.000000,0.0000,...,0,0,0,1,0,0,0,1,0,0
3,6,0.695312,0.000000,0.783359,0.185547,0.088281,0.104375,0.09375,0.062500,0.0000,...,1,0,0,1,0,0,0,1,0,0
4,3,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,1.37793,0.000244,0.1250,...,0,0,0,1,0,0,0,0,0,1


In [61]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

# Feature selection with Lasso
X = df.drop('B_Reach_cms', axis=1)
y = df['B_Reach_cms']

# Create the KNN model
lasso = Lasso(random_state=42)

lasso.fit(X, y)

# Get features with non-zero weights
non_zero_features = np.where(lasso.coef_ != 0)[0]

# Display the feature indices with non-zero weights
print("Features with non-zero weights:", non_zero_features)

Features with non-zero weights: [ 16  17  18  25  35  39  41  45  47  48  49  64  65  80  82  88  89 102
 103 112 113 114 115 129 130 131]


In [62]:
# Define the parameter grid
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0, 5.0],
    'fit_intercept': [True, False],
    'precompute': [True, False],
    'max_iter': [1000, 2000, 5000],
    'tol': [0.0001, 0.001, 0.01],
    'selection': ['cyclic', 'random']
}

# Create the GridSearchCV object
grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit the data to perform grid search
grid_search.fit(X, y)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best R-squared Score:", grid_search.best_score_)

/Users/azka/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 46285.067682337714, tolerance: 14637.032661359981
  model = cd_fast.enet_coordinate_descent_gram(
/Users/azka/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 45590.012535298134, tolerance: 14646.935867359982
  model = cd_fast.enet_coordinate_descent_gram(
/Users/azka/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 46843.86631145469, tolerance: 14648.893282799982
  model = cd_fast.enet_coordinate_descent_gram(
/Users/azka/opt/anaconda3/lib/python3.9/site-packages/sk

Best Parameters: {'alpha': 0.1, 'fit_intercept': True, 'max_iter': 1000, 'precompute': False, 'selection': 'random', 'tol': 0.01}
Best R-squared Score: 0.7982706857746097


In [63]:
# Feature importance
feature_importance = grid_search.best_estimator_.coef_
print(feature_importance)

# Get the indices of the feature importance
feature_importance_indices = np.argsort(feature_importance)

[ 5.49735674e-01  0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00  5.65980030e-04
  7.81171409e-04 -4.28276360e-04  1.70709603e-03  4.15895909e-04
 -8.67692743e-03  2.42502502e-03 -1.25407738e-02  0.00000000e+00
  0.00000000e+00  1.04036932e-02  0.00000000e+00  6.04930326e-03
 -1.05392100e-02 -1.04602075e-02  1.37433470e-02  1.07664735e-02
 -0.00000000e+00  0.00000000e+00 -5.02729006e-03 -2.35280035e-02
 -0.00000000e+00  3.83352562e-02 -0.00000000e+00  8.63658214e-04
  3.35591722e-03  8.17607885e-04 -3.37065808e-04 -6.84991253e-02
  7.10639566e-02 -2.38571096e-02 -0.00000000e+00 -1.00266320e-03
 -5.21047898e-03  0.00000000e+00  3.09032423e-02  1.24316045e-03
  6.82246545e-04  7.13700486e-04  7.98143816e-04 -0.00000000e+00
  3.92223824e-02  0.00000000e+00  2.14721002e-02  8.51445859e-03
 -0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000

In [64]:
print(len(feature_importance_indices))

148


In [65]:
non_zero_features = np.where(feature_importance != 0)[0]

non_zero_features_indices = np.argsort(feature_importance[non_zero_features])

print(len(non_zero_features_indices))

69


In [66]:
# df_feature_selection is df that has column with non-zero weight
df_feature_selection = df.iloc[:, non_zero_features]

df_feature_selection.head()

,weight_class,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_opp_SIG_STR_att,B_avg_opp_SIG_STR_landed,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,B_avg_opp_TD_att,...,R_avg_opp_CTRL_time(seconds),R_total_time_fought(seconds),R_total_rounds_fought,R_total_title_bouts,R_win_by_Decision_Unanimous,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Weight_lbs,B_age,R_age
0,7,42.000000,29.625000,94.375000,29.250000,77.000000,46.000000,96.000000,60.625000,3.537109,...,117.769531,900.000000,0.0,0.0,0.0,0.0,182.88,185.0,26.0,21.0
1,5,137.460938,53.234375,89.953125,31.296875,141.210938,56.718750,91.882812,33.078125,1.835938,...,115.000000,890.593750,17.0,0.0,4.0,0.0,180.34,170.0,27.0,30.0
2,6,49.968750,36.000000,78.375000,33.605469,84.500000,37.000000,96.000000,57.500000,1.500000,...,79.500000,598.500000,0.0,0.0,0.0,0.0,185.42,170.0,28.0,31.0
3,6,39.218750,19.015625,37.226562,14.531250,51.500000,30.742188,52.046875,28.671875,1.625000,...,116.292816,793.567753,44.0,1.0,5.0,0.0,177.80,170.0,28.0,34.0
4,3,74.522461,29.986572,93.299805,42.413086,102.170410,51.280762,125.859863,71.359619,3.682373,...,162.796875,1129.593750,22.0,3.0,2.0,1.0,160.02,135.0,34.0,31.0


In [67]:
df_feature_selection['B_Reach_cms'] = y

/var/folders/53/kv053qr13vxg2_y_1lg6hjg80000gn/T/ipykernel_52342/3603037234.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feature_selection['B_Reach_cms'] = y


In [68]:
# Export to CSV
df_feature_selection.to_csv('../Punya Andi/UFC_kombinasi5_FS_lasso.csv', index=False)
